In [12]:
using CSV
import Pkg
Pkg.add("JSON3")
using JSON3
using DataFrames


# Read the Locations CSV file
df = CSV.read("Locations_Capabilities.csv", DataFrame)

# Function to parse JSON strings into Julia objects
function parse_json_string(str)
    # Replace single quotes with double quotes for valid JSON
    str = replace(str, "'" => "\"")
    return JSON3.read(str)
end

# Process each location's data
locations_data = Dict()

for row in eachrow(df)
    location_id = row.Location_ID
    
    # Process workers data
    workers = parse_json_string(row.Workers)
    
    # Process machines data
    machines = parse_json_string(row.Machines)
    
    # Process inventory resources
    inventory = parse_json_string(row.Inventory_Resources)
    
    # Store all data for this location
    locations_data[location_id] = Dict(
        "workers" => workers,
        "machines" => machines,
        "inventory" => inventory
    )
end

# Create helper functions
function get_all_location_ids()
    return locations_data.keys()
end

function get_all_workers_ids(location_id)
    return locations_data[location_id]["workers"].keys()
end

function get_worker_capabilities(location_id, worker_id)
    return locations_data[location_id]["workers"][worker_id]["Capabilities"]
end

function get_all_machines_ids(location_id)
    return locations_data[location_id]["machines"].keys()
end

function get_machine_capabilities(location_id, machine_id)
    return locations_data[location_id]["machines"][machine_id]["Capabilities"]
end

function get_all_resources_ids(location_id)
    return locations_data[location_id]["inventory"].keys()
end

function get_resource_amounts(location_id, resource_id)
    return locations_data[location_id]["inventory"][resource_id]
end

println(locations_data["L1"]["machines"][1])


   Resolving package versions...
  No Changes to `C:\Users\gcpol\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\gcpol\.julia\environments\v1.11\Manifest.toml`


{
       "Machine_ID": "L1_M1",
     "Capabilities": "Painting",
   "Breakdown_Prob": 0.11
}


In [14]:
using CSV
using JSON3
using DataFrames

# Read the CSV file
df = CSV.read("Projects_Data_with_Stage_Durations (1).csv", DataFrame)

# Function to clean and parse JSON strings into Julia objects
function parse_json_string(str)
    # Clean the string: remove newlines and extra spaces
    str = replace(str, r"\s+" => " ")  # Replace multiple spaces with single space
    str = replace(str, "\n" => "")     # Remove newlines
    str = strip(str)                   # Remove leading/trailing whitespace
    
    # Replace single quotes with double quotes for valid JSON
    str = replace(str, "'" => "\"")
    
    try
        return JSON3.read(str)
    catch e
        println("Error parsing JSON string: ", str)
        rethrow(e)
    end
end

# Process each project's data
projects_data = Dict()

for row in eachrow(df)
    project_id = strip(row.Project_ID)
    
    # Process all JSON fields
    total_resources = parse_json_string(row.Total_Resources)
    workers_req = parse_json_string(row.Workers_Requirement)
    machines_req = parse_json_string(row.Machines_Requirement)
    stages = parse_json_string(row.Stages)
    
    # Store all data for this project
    projects_data[project_id] = Dict(
        "total_resources" => total_resources,
        "workers_requirement" => workers_req,
        "machines_requirement" => machines_req,
        "deadline" => row.Deadline,
        "customer_priority" => row.Customer_Priority,
        "stages" => stages,
        "revenue" => row.Revenue
    )
end

# Create helper functions
function get_project_data(project_id)
    return projects_data[project_id]
end

function get_all_project_ids()
    return collect(keys(projects_data))
end

function get_project_stages(project_id)
    return projects_data[project_id]["stages"]
end

function get_project_resources(project_id)
    return projects_data[project_id]["total_resources"]
end

function get_project_stage_workers(project_id, stage_id)
    stages = projects_data[project_id]["stages"]
    return stages[stage_id]["Workers_Needed"]
end

function get_project_stage_machines(project_id, stage_id)
    stages = projects_data[project_id]["stages"]
    return stages[stage_id]["Machines_Needed"]
end

function get_project_stage_duration(project_id, stage_id)
    stages = projects_data[project_id]["stages"]
    return stages[stage_id]["Duration_days"]
end

function get_project_deadline(project_id)
    return projects_data[project_id]["deadline"]
end

function get_project_priority(project_id)
    return projects_data[project_id]["customer_priority"]
end

function get_project_revenue(project_id)
    return projects_data[project_id]["revenue"]
end

projects = get_all_project_ids()

project_stages = Dict(
    p => collect(1:length(get_project_stages(p)))
    for p in projects
)

 

# Print some data to verify it worked
first_project_id = get_all_project_ids()[1]
second_project_id = get_all_project_ids()[2]
println("First project data:")
println("Project ID: ", first_project_id)
println("Deadline: ", get_project_deadline(first_project_id))
println("Priority: ", get_project_priority(first_project_id))
println("Revenue: ", get_project_revenue(first_project_id))
println("Total Resources: ", get_project_resources(first_project_id))
println("First Stage: ", get_project_stages(first_project_id)[1])
println("Second Stage: ", get_project_stages(second_project_id)[1])

First project data:
Project ID: P13
Deadline: 66
Priority: 1
Revenue: 16000
Total Resources: {
   "Plastic": 66,
     "Steel": 51,
     "Paint": 35
}
First Stage: {
           "Stage_ID": 1,
    "Machines_Needed": [
                         "Inspector"
                       ],
     "Workers_Needed": [
                         "Painter",
                         "Inspector"
                       ],
   "Resources_Needed": {
                          "Plastic": 47,
                            "Steel": 27,
                            "Paint": 13
                       },
      "Duration_days": 8
}
Second Stage: {
           "Stage_ID": 1,
    "Machines_Needed": [
                         "Assembler",
                         "Packer"
                       ],
     "Workers_Needed": [
                         "Painter"
                       ],
   "Resources_Needed": {
                          "Plastic": 14,
                            "Steel": 23,
                            "Paint": 20

In [15]:
#Preparing the rest of the Parameters

import Pkg; Pkg.add("GLPK")
using JuMP, GLPK, DataFrames, CSV

########################################################
# Important Parameter, this is the end of the job scheduler
deadline = 100
########################################################

# Get all locations
locations = collect(keys(locations_data))

# Create location-specific sets for workers and machines
location_workers = Dict(
    loc => collect(1:length(locations_data[loc]["workers"]))
    for loc in locations
)

location_machines = Dict(
    loc => collect(1:length(locations_data[loc]["machines"]))
    for loc in locations
)

# Get all projects and their stages
projects = get_all_project_ids()
project_stages = Dict(
    p => collect(1:length(get_project_stages(p))) 
    for p in projects
)

T = 1:deadline




   Resolving package versions...
  No Changes to `C:\Users\gcpol\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\gcpol\.julia\environments\v1.11\Manifest.toml`


1:100

In [8]:
using JuMP, Gurobi, JSON3, DataFrames

# --- Data Loading ---
function load_data()
    # Load projects data
    df_projects = CSV.read("Projects_Data_with_Stage_Durations (1).csv", DataFrame)
    projects_data = Dict()
    for row in eachrow(df_projects)
        project_id = strip(row.Project_ID)
        projects_data[project_id] = Dict(
            "stages" => JSON3.read(replace(row.Stages, "'" => "\"")),
            "deadline" => row.Deadline,
            "revenue" => row.Revenue
        )
    end

    # Load locations data
    df_locations = CSV.read("Locations_Capabilities.csv", DataFrame)
    locations_data = Dict()
    for row in eachrow(df_locations)
        location_id = row.Location_ID
        locations_data[location_id] = Dict(
            "workers" => JSON3.read(replace(row.Workers, "'" => "\"")),
            "machines" => JSON3.read(replace(row.Machines, "'" => "\""))
        )
    end
    return projects_data, locations_data
end

projects_data, locations_data = load_data()


get_all_project_ids() = collect(keys(projects_data))
get_project_stages(p) = projects_data[p]["stages"]
get_project_deadline(p) = projects_data[p]["deadline"]
get_project_revenue(p) = projects_data[p]["revenue"]
get_project_stage_duration(p,s) = projects_data[p]["stages"][s]["Duration_days"]

get_all_location_ids() = collect(keys(locations_data))
get_all_workers_ids(l) = collect(keys(locations_data[l]["workers"]))
get_all_machine_ids(l) = collect(keys(locations_data[l]["machines"]))
get_worker_capabilities(l,w) = get(locations_data[l]["workers"][w], "capabilities", ["basic"])
get_machine_capabilities(l,m) = get(locations_data[l]["machines"][m], "capabilities", ["general"])

#Model
model = Model(Gurobi.Optimizer)

#Parameters
projects = get_all_project_ids()
locations = get_all_location_ids()
T = 1:maximum(get_project_deadline(p) for p in projects)
worker_capacity = 3 

#Variables
@variable(model, x[p in projects], Bin) 
@variable(model, stage_start[p in projects, s in keys(get_project_stages(p))], Int)
@variable(model, z[p in projects, s in keys(get_project_stages(p)), 
                  l in locations, w in get_all_workers_ids(l), 
                  m in get_all_machine_ids(l), t in T], Bin)


##Must select at least 1 project
@constraint(model, sum(x[p] for p in projects) >= 1)

##Project scheduling
for p in projects
    stages = keys(get_project_stages(p))
    # Link stages
    for (i,s) in enumerate(sort(collect(stages)))
        if i > 1
            prev_s = sort(collect(stages))[i-1]
            @constraint(model, stage_start[p,s] >= stage_start[p,prev_s] + 
                       get_project_stage_duration(p,prev_s))
        end
        # Deadline constraint
        @constraint(model, stage_start[p,s] + get_project_stage_duration(p,s) <= get_project_deadline(p))
    end
end

## Resource constraints
for l in locations, t in T
    # Worker capacity
    @constraint(model, sum(z[p,s,l,w,m,t] for p in projects, 
                          s in keys(get_project_stages(p)),
                          w in get_all_workers_ids(l),
                          m in get_all_machine_ids(l)) <= worker_capacity)
    
    # Capability matching
    for p in projects, s in keys(get_project_stages(p))
        req = get_project_stages(p)[s]
        for w in get_all_workers_ids(l)
            if !isempty(intersect(get(req, "Workers_Needed", []), 
                                 get_worker_capabilities(l,w)))
                for m in get_all_machine_ids(l)
                    if !isempty(intersect(get(req, "Machines_Needed", []), 
                                         get_machine_capabilities(l,m)))
                        @constraint(model, z[p,s,l,w,m,t] <= x[p])
                    else
                        @constraint(model, z[p,s,l,w,m,t] == 0)
                    end
                end
            else
                @constraint(model, [m in get_all_machine_ids(l)], z[p,s,l,w,m,t] == 0)
            end
        end
    end
end

# Objective
@objective(model, Max, sum(get_project_revenue(p) * x[p] for p in projects))
-
optimize!(model)



#Verification
function safe_get(var)
    try
        return value(var)
    catch
        return 0.0
    end
end

println("\n=== RESULTS ===")
accepted = [p for p in projects if safe_get(x[p]) > 0.5]
if isempty(accepted)
    println("No projects selected! Check constraints and:")
    println("- Resource capacities (current worker capacity = $worker_capacity)")
    println("- Project deadlines vs stage durations")
    println("- Capability requirements")
else
    println("Accepted Projects (", length(accepted), "):")
    for p in accepted
        println("- $p (Revenue: ", get_project_revenue(p), ")")
        for s in sort(collect(keys(get_project_stages(p))))
            start_t = safe_get(stage_start[p,s])
            println("  Stage $s starts at day $start_t")
        end
    end
    println("\nTotal Revenue: ", sum(get_project_revenue(p) for p in accepted))
end

Set parameter Username
Set parameter LicenseID to value 2651032
Academic license - for non-commercial use only - expires 2026-04-14
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 597017 rows, 596771 columns and 1193533 nonzeros
Model fingerprint: 0xf0f3e7aa
Variable types: 0 continuous, 596771 integer (596720 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 4e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 452500.00000
Presolve removed 597017 rows and 596771 columns
Presolve time: 0.38s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.60 seconds (0.32 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: